In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

# Extracting features and target variable
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)

# Define the Decision Tree Classifier model
class DecisionTree(nn.Module):
    def __init__(self, input_size, output_size):
        super(DecisionTree, self).__init__()
        self.layer = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = torch.sigmoid(self.layer(x))
        return x

# Instantiate the model
input_size = X_train.shape[1]
output_size = 1  # binary classification
model = DecisionTree(input_size, output_size)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training the model
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, Y_train.float().view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    # Predictions on the test set
    outputs = model(X_test)
    predictions = (outputs > 0.5).int().view(-1)

    # Confusion matrix
    cm = torch.zeros(2, 2, dtype=torch.int64)
    for i in range(len(Y_test)):
        cm[Y_test[i], predictions[i]] += 1

    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    accuracy = (predictions == Y_test).sum().item() / len(Y_test)
    print(f"Accuracy: {accuracy:.2f}")


Epoch [100/1000], Loss: 0.6472
Epoch [200/1000], Loss: 0.5655
Epoch [300/1000], Loss: 0.5152
Epoch [400/1000], Loss: 0.4822
Epoch [500/1000], Loss: 0.4596
Epoch [600/1000], Loss: 0.4433
Epoch [700/1000], Loss: 0.4312
Epoch [800/1000], Loss: 0.4219
Epoch [900/1000], Loss: 0.4145
Epoch [1000/1000], Loss: 0.4087
Confusion Matrix:
tensor([[65,  3],
        [ 6, 26]])
Accuracy: 0.91
